In [17]:
%%bash

pip install --upgrade pip

pip install langchain
pip install faiss-cpu
pip install unstructured
pip install sentence_transformers

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


https://huggingface.co/learn/cookbook/en/advanced_rag

In [4]:
from langchain_community.embeddings import OllamaEmbeddings

from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader('data/wiki_gameofthrones_txt1/', glob="**/*.txt")

docs = loader.load()
len(docs)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/addo/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


183

In [18]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

/home/addo/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
.gitattributes: 100%|██████████| 1.52k/1.52k [00:00<00:00, 7.62MB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 539kB/s]
README.md: 100%|██████████| 90.8k/90.8k [00:00<00:00, 806kB/s]
config_sentence_transformers.json: 100%|██████████| 124/124 [00:00<00:00, 549kB/s]
model.safetensors: 100%|██████████| 133M/133M [00:01<00:00, 96.6MB/s] 
pytorch_model.bin: 100%|██████████| 134M/134M [00:01<00:00, 84.1MB/s] 
sentence_bert_config.json: 100%|██████████| 52.0/52.0 [00:00<00:00, 102kB/s]
special_tokens_map.json: 100%|██████████| 125/125 [00:00<00:00, 250kB/s]
tokenizer.json: 100%|██████████| 711k/711k [00:00<00:00, 1.57MB/s]
tokenizer_config.json: 100%|██████████| 366/366 [00:00<00:00, 912kB/s]
vocab.txt: 100%|██████████| 232k

384

In [40]:
from langchain_community.vectorstores.utils import DistanceStrategy

MARKDOWN_SEPARATORS = [
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    "=="
    "==="
    " ",
    "",
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=10,  # the maximum number of characters in a chunk: we selected this value arbitrarily
    chunk_overlap=1,  # the number of characters to overlap between chunks
    # add_start_index=True,  # If `True`, includes chunk's start index in metadata
    # strip_whitespace=True,  # If `True`, strips whitespace from the start and end of every document
    # separators=MARKDOWN_SEPARATORS,
)

documents = text_splitter.split_documents(docs)

KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(
    docs, hf, distance_strategy=DistanceStrategy.COSINE
)

In [41]:
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query="Who is the father of Arya Stark?", k=5)
for i in range(0,5):
    print(retrieved_docs[i].page_content)
    print("--metadata--")
    print(retrieved_docs[i].metadata)
    print("=============================================================================")
    print("=============================================================================")

'''Arya Stark''' is a fictional character in American author George R. R. Martin's ''A Song of Ice and Fire'' epic fantasy novel series. She is a prominent point of view character in the novels with the third most viewpoint chapters, and is the only viewpoint character to have appeared in every published book of the series.

Introduced in 1996's ''A Game of Thrones'', Arya is the third child and younger daughter of Lord Eddard Stark and his wife Lady Catelyn Stark. She is tomboyish, headstrong, feisty, independent, disdains traditional female pursuits, and is often mistaken for a boy. She wields a smallsword named Needle, a gift from her half-brother, Jon Snow, and is trained in the Braavosi style of sword fighting by Syrio Forel.

Arya is portrayed by English actress Maisie Williams in HBO's Emmy-winning television adaptation of the novel series, ''Game of Thrones''. Her performance has garnered critical acclaim, particularly in the second season for her work opposite veteran actor Ch